# Julia: A Practical Introduction
Here we provide a brief tutorial on coding in Julia, focusing on aspects that are helpful for working with models in `JuMP.jl` and `InfiniteOpt.jl`. As such, this is not intended as a comprehensive guide to coding in Julia. Note that this tutorial draws inspiration from the content provided in https://jump.dev/JuMP.jl/dev/tutorials/getting_started/getting_started_with_julia/#Getting-started-with-Julia.

## Resources
- Julia's compendium of learning resources: https://julialang.org/learning/
- Julia's documentation: https://jump.dev/JuMP.jl/stable/
- Julia community forum: https://discourse.julialang.org/
- The help interface in the REPL (accessed via `?` followed by the function name of interest)

## Arithmetic

With optimization in mind, we'll be dealing with a lot of mathematical operations (arithmetic). Julia follows a straightforward syntax that is similar to MATLAB.

In [ ]:
@show 1 + 1
@show 1 - 2
@show 2 * 2
@show 4 / 5
@show 3^2;

Like MATLAB and python, Julia is a dynamic language, so we don't have to declare the types of variables explicitly. Notice how some of the above results don't end in `.0`. (Note that `@show` is a macro that prints code and the output, macros are special kinds of functions that begin with `@`.) We can check the types using the `typeof` function.

In [ ]:
@show typeof(1)
@show typeof(1.0);

Here we have an `Int64` and a `Float64` which denote an integer and a floating point number, respectively, each with 64-bits of precision. These are the numerical types we will encounter most often in formulating optimization problems. 

Julia also provides a suite of common mathematical functions such as those featured below. A complete list is provided at https://docs.julialang.org/en/v1/base/math/#Mathematical-Functions.

In [ ]:
@show sin(0.5π)
@show cos(0)
@show tanh(0)
@show log(10)
@show exp(3)
@show sqrt(4) == √4; # create `√` via \sqrt and then pressing [tab]

### Floating Point Numbers
Before moving on we should emphasize that floating point numbers are an approximation of real valued numbers. This means that computer arithmetic with floating point number is not always exactly the same as the pure mathematical version. For example:

In [ ]:
@show 0.1 * 3 == 0.3
@show 0.1 * 3 - 0.3 # the difference
@show sin(2π / 3) == √3 / 2
@show sin(2π / 3) - √3 / 2; # the difference

The error is small, but nonzero. One way of explaining this difference is to consider how we would write $\frac{1}{3}$ and $\frac{2}{3}$ using only four digits after the decimal point. We would write $\frac{1}{3} = 0.3333$ and $\frac{2}{3} = 0.6667$. So, despite the fact that $2 \frac{1}{3} = \frac{2}{3}$, `2 * 0.3333 == 0.6666 != 0.6667`.

For better comparisons, we can use `≈` (\approx + [tab]) instead of `==`:

In [ ]:
@show 0.1 * 3 ≈ 0.3
@show sin(2π / 3) ≈ √3 / 2;

Here `≈` is just a convenient syntax for the `isapprox` function which compares floating point number relative to a certain tolerance `atol`.

In [ ]:
isapprox(0.1 * 3, 0.3, atol = 1e-8) # syntax `1e-8` is the same as `1 * 10^-8`

This motivates the use of numerical tolerances in optimization solvers. For instance, the value of a binary variable may not be exactly equal to `1` or `0`.

Some illustrative pitfalls are:

In [ ]:
@show 1 + 1e-16 == 1 # adding 2 values with very different orders of magnitude
@show (1 + 1e-16) - 1e-16 == 1 + (1e-16 - 1e-16); # floating point arithmetic is not associative

This is not a Julia-specific issue, but is true of every programming language.

## Julia Variables
As we start to build scripts of operations, we will typically store the results in a Julia variable. A Julia variable will store the value of a number (or other data type as we discuss further below). This is accomplished in like manner to other dynamic languages like python and MATLAB. 

In [ ]:
x = 42
sin(x)

This is a simple concept, but it is important to make this distinction now before we start talking about the decision variables associated with `JuMP.jl` and `InfiniteOpt.jl` models. 

Another cool thing we can do in Julia is use unicode characters as variables. These can be inserted by typing `\name` and then pressing [TAB]. A list of supported characters is provided at https://docs.julialang.org/en/v1/manual/unicode-input/#Unicode-Input.

In [ ]:
λ = 1e-4
δ = 0.1
🐟 = 42
🚚 = 10;

## Vectors, Matrices, and Arrays
Like MATLAB, Julia supports array objects (i.e., vectors in 1D and matrices in 2D). The simplest way to create a vector is using comma-separated elements in square brackets:

In [ ]:
b = [5, 6]

Matrices can be made with spaces separating columns and semicolons separating rows:

In [ ]:
A = [1.0 2.0; 3.0 4.0]

Note that like MATLAB (and unlike python) these arrays are 1-indexed. We can extract particular value(s) from the arrays by indexing them with bracketed values:

In [ ]:
@show b[1] # access the first element of b
@show A[2, 1] # access the second row and first column of A 
@show A[:, 1] # access the first column of A 
@show A[2, 2:end]; # access the second row from the 2nd column to the last column

Notice that when we index with `:` we end up getting back another array instead of a single value. This is called array slicing (we are accessing some sliced portion of it).

### Array Arithmetic
Like MATLAB, the usual operators respect the dimensions of the arrays to do arithmetic that is consistent with linear algebra.

In [ ]:
@show x = A \ b # solve for x in A * x = b
@show A * x # matrix-vector multiplication
@show b - b
@show A + 2A
@show b' * b # inner product 
@show b * b'; # outer product

We can also do element-wise operations by adding a `.` in front of arithmetic operators in like manner to MATLAB:

In [ ]:
@show b .* b
@show A ./ A 
@show b .^ 2;

## Comparisons and Logical Operators
As we work with numbers/arrays, we will commonly want to compare their values. Here the syntax is similar MATLAB:

In [ ]:
result = 1 <= 3
@show typeof(result)
@show 1 == 1
@show 1 >= 3
@show 1 < 1
@show 2 > -1
@show 1 != 3
@show b .<= 5; # apply comparisons element-wise for arrays

Moreover, we can aggregate boolean results with logical operators:

In [ ]:
@show 1 <= 3 && 4 != 2 # logical and
@show 1 <= 3 || 1 != 1 # logical or
@show !(1 < 3); # negation

## Other Common Types
In addition to numbers and arrays, there are a few other types we will commonly use to define optimization models.

### Strings
These store text and are defined using double quotes (not single quotes like python):

In [ ]:
typeof("I love optimization!")

We can also embed unicode characters

In [ ]:
my_str = "I ❤ optimization!"

We can print strings using `println` which will combine strings together if we give multiple:

In [ ]:
x = 42
println("The answer to life, the universe, and everything is ", x, "!") # we can embed numbers as individual arguments

`String`s can be indexed and sliced like vectors. However, unlike python, Julia `String`s are not mutable. 

In [ ]:
@show my_str[1:2]
my_str[8] = "s" # throws an error since we cannot modify a String in-place

### Symbols
`Symbol`s are a special Julia type that is uses internally to store names (these are used a lot with metaprogramming). In this introductory context, symbols can serve as labels that use less memory than `String`s. Note however, that Symbols are often misused where `@enum` or a `String` should be used instead. 

In [ ]:
typeof(:x)

We can convert `Symbol`s into `String`s and vice versa:

In [ ]:
@show String(:x)
@show Symbol("x");

### Tuples
`Tuple`s are a memory efficient way of storing things that Julia uses a lot. A `Tuple` is am immutable collection of values. For instance,

In [ ]:
t = (42, "cat", :x)
typeof(t)

These are indexed just like vectors:

In [ ]:
t[2]

We can also unpack them:

In [ ]:
a, b, c = t # define a variable for each tuple element
println("a = ", a, ", b = ", b, ", c = ", c)

We can also light-weight data structures called named tuples:

In [ ]:
t = (word = "hello", num = 1.2, sym = :foo)
t.word # access the names via the dot syntax

### Dictionaries
Like python, Julia supports dictionaries. Dictionaries are a very useful tool for storing data that can be accessed with arbitrary keys to values. For instance, mapping strings to integers:

In [ ]:
d = Dict("a" => 1, "b" => 2, "c" => 3)

We can access dictionary values with square bracket indexing, and we can change/add values.

In [ ]:
@show d["a"]
d["b"] = 42 # change the value of "b"
d["d"] = -2 # add a new mapping "d" => -2
d

Dictionaries can stored varied input, including other dictionaries:

In [ ]:
d2 = Dict("a" => 1, "b" => 2, "c" => Dict(:x => 3, :y => 4))

This gives a very flexible way to store data. Notice however that now the supports values of type `Any` which allows it to hold heterogeneous values, but this will be less performant that dictionaries where the value types match.

### Structs
We can define our own custom data structure via `struct`:

In [ ]:
struct MyStruct
    x::Int
    y::String
    z::Dict{Int,Int}
end

a = MyStruct(1, "a", Dict(2 => 3))

The fields are accessed via dot syntax, but these are not mutable by default:

In [ ]:
@show a.x
a.x = 2 # throws an error

However, we can make a mutable `struct` by adding the `mutable` keyword: 

In [ ]:
mutable struct MyStructMutable
    x::Int
    y::String
    z::Dict{Int,Int}
end

a = MyStructMutable(1, "a", Dict(2 => 3))
a.x = 2

Notice in the above examples, that we explicitly typed each field. This is optional but highly recommended for performance. See https://docs.julialang.org/en/v1/manual/types/#Composite-Types and https://docs.julialang.org/en/v1/manual/performance-tips/#Type-declarations for more information.

## Loops
Julia supports for loops and while loops. We will focus on the use of for loops which are of the form `for <value> in <collection> end`. A common collection (i.e., iterator) is a range which is formatted `<start>:<end>` or `<start>:<step>:<end>`. For example,

In [ ]:
for i in 1:5
    println(i)
end

for i in 1.2:1.1:5.6
    println(i)
end

This also works with dictionaries




In [ ]:
for (key, value) in Dict("A" => 1, "B" => 2.5, "C" => 2 - 3im)
    println(key, " : ", value)
end

Iterating over arrays is also common and easy

In [ ]:
A = zeros(2, 4)
for j in 1:4
    for i in 1:2
        A[i, j] = i + j
    end
end 
A

Unlike languages like MATLAB, R, and python, for loops are actually performant in Julia. There is no need to call a lot of convoluted numpy functions for performance. 

## Control Flow
Julia control flow is similar to MATLAB, using keywords `if-elseif-else-end`:

In [ ]:
for i in 0:5:15
    if i < 5
        println(i, " is less than 5")
    elseif i < 10
        println(i, " is less than 10")
    else
        if i == 10
            println("the value is 10")
        else
            println(i, " is bigger than 10")
        end
    end
end

## Comprehensions
Similar to languages like Haskell and Python, Julia supports the use of simple loops in the construction of arrays and dictionaries, called comprehensions. 

A list of increasing integers:

In [ ]:
[i for i in 1:3]

Matrices can be built by including multiple indices:

In [ ]:
[i * j for i in 1:3, j in 5:10]

Conditional statements can be used to filter out some values:

In [ ]:
[i for i in 1:5 if i % 2 == 1] # the `%` operator is the modulo operator in Julia

A similar syntax can be used for building dictionaries:

In [ ]:
Dict(string(i) => i for i in 1:10 if i % 2 == 1)

## Functions
Functions are declared using the `function` keyword and the final value (if any) is returned via the `return` keyword. Let's begin with a simple function:

In [ ]:
function say_hi()
    return println("Hi there!")
end
say_hi()

We can also add arguments:

In [ ]:
function my_square(x)
    return x^2
end
my_square(2)

We can even add optional keyword arguments if we want:

In [ ]:
function mult(x; y = 2.0)
    return x * y
end
@show mult(4.0)
@show mult(4.0, y = 5.0);

### Anonymous Function
The syntax `input -> output` creates an anonymous function. These are most useful when passed to other functions. For example:

In [ ]:
f = x -> x^2
f(2)

In [ ]:
findall(x -> x >= 0, [0, -3, 4, 8, -9]) # find all the indices of the nonnegative values

### Typed Functions
We can constrain the inputs to a function using type parameters, which are :: followed by the type of the input we want. This allows us to overload a function based on type and is a core programming paradigm used in Julia. For example:

In [ ]:
function g(x::Int)
    return x^2
end

function g(x::Float64)
    return exp(x)
end

function g(x::Number)
    return x + 1
end

@show g(2)
@show g(2.0)
@show g(1 + 1im);

But what happens if we call `foo` with something we haven't defined it for?

In [ ]:
g([1, 2, 3])

We get a dreaded `MethodError`! A `MethodError` means that you passed a function something that didn't match the type that it was expecting. In this case, the error message says that it doesn't know how to handle a `Vector{Int64}`, but it does know how to handle `Float64`, `Int64`, and `Number`.

### Broadcasting
In the example above, we didn't define what to do if f was passed a `Vector`. Luckily, Julia provides a convenient syntax for mapping f element-wise over arrays! Just add a `.` between the name of the function and the opening `(`. This works for any function, including functions with multiple arguments. For example:

In [ ]:
g.([1, 2, 3])

This will be a common paradigm when dealing with arrays in `JuMP.jl` and `InfiniteOpt.jl`.

### Mutable Arguments
Some types in Julia are *mutable*, which means you can change the values inside them. A good example is an array. You can modify the contents of an array without having to make a new array.

In contrast, types like `Float64` are immutable. You can't modify the contents of a `Float64`.

This is something to be aware of when passing types into functions. For example:

In [ ]:
function mutability_example(mutable_type::Vector{Int}, immutable_type::Int)
    mutable_type[1] += 1
    immutable_type += 1
    return
end

mutable_type = [1, 2, 3]
immutable_type = 1

mutability_example(mutable_type, immutable_type)

@show mutable_type
@show immutable_type;

Because `Vector{Int}` is a mutable type, modifying the variable inside the function changed the value outside the function. In contrast, the change to `immutable_type` didn't modify the value outside the function. We can check mutability via `isimmutable`:

In [ ]:
@show isimmutable(mutable_type)
@show isimmutable(immutable_type);

Note that Julia functions that change the inputs are typically annotated with an `!` at the end of the function name (e.g., `append!`). However, functions obviously mutate their input in `JuMP.jl` and `InfiniteOpt.jl` (e.g., `JuMP.delete`) drop this convention. 

## Package Manager

### Installing Packages
No matter how wonderful Julia's base language is, at some point you will want to use an extension package. Some of these are built-in, for example random number generation is available in the Random package in the standard library. These packages are loaded with the commands `using` and `import`.

In [ ]:
using Random  # The equivalent of Python's `from Random import *`
import Random  # The equivalent of Python's `import Random`

Random.seed!(33)

[rand() for i in 1:4]

The Package Manager is used to install packages that are not part of the standard library. This is accomplished using the `add` command. I recommend doing this via the REPL. For more information see https://pkgdocs.julialang.org/v1/getting-started/.

```julia
julia> ]

(@v1.7) pkg> add InfiniteOpt
```

### Virtual Environments
By default, packages are added in Julia's global environment. Installing a wide variety of packages can lead to compatibility problems. Hence, we highly recommend creating a new environment for each project you work on. Note that this very short course has its own environment as directed by the `Project.toml` file.

```julia
julia> pwd()
/Users/MyAccount

julia> mkdir("MyProject")

julia> cd("MyProject")
/Users/MyAccount/MyProject

julia> ]

(@v1.7) pkg> activate .

(@MyProject) pkg> add InfiniteOpt
```

For more information, see https://pkgdocs.julialang.org/v1/environments/.

## Latency in Julia
As you may have already noticed, the first time we run code in Julia it isn't very fast. This is because functions need to be compiled the first time they are called. From then on, they will be much faster. In many applications, this isn't a big problem. However, sometimes this can be a barrier. Here we will discuss 2 ways to avoid this.

### Don't use the command line
In other languages, you might be used to the workflow:
```
$ julia my_script.jl
```
This doesn't work well for Julia since we will have to pay the compilation latency every time. Instead, it is better to do repeated calls in the same REPL session (VS Code does this automatically).

### Use PackageCompiler
In cases where we would like to remove the latency completely, we can use `PackageCompiler.jl` to cache the compilation for future runs and thus avoid most of the start-up latency. For instance, if we have a script `model.jl` that uses `JuMP.jl` and `HiGHS.jl` we can do the following:

```julia
using PackageCompiler, Libdl
PackageCompiler.create_sysimage(
    ["JuMP", "HiGHS"],
    sysimage_path = "customimage." * Libdl.dlext,
    precompile_execution_file = "model.jl",
)
```

Then we can use `customimage` when we run `model.jl` and get a great speedup:
```
$ time julia model.jl
15.78s user 0.48s system 100% cpu 16.173 total

$ time julia --sysimage customimage model.jl
0.68s user 0.22s system 153% cpu 0.587 total
```

We only need to build `customimage` once, and it can even be used with other Julia scripts to remove some latency. Note that system images are system specific and cannot be transferred between computers.

## Exercise: Monte Carlo Pi
**Problem**
- Estimate the value of π using Monte Carlo sampling
- Estimate π using 100, 1000, 10000, and 100000 samples

**Algorithm**
- Generate random 2D points between in the interval [-1, 1] x [-1, 1]
- Determine the number of points inside the unit circle
- The estimate is $\pi = 4 \frac{\text{number of points in the circle}}{\text{total number of points}}$

**Hints**
- A random 2D vector in [-1, 1] x [-1, 1] is `rand(2) * 2 .- 1`
- Should put code inside a function that takes the number of samples as input



In [ ]:
# PUT SOLUTION HERE
